# Likelihood Analysis with Python

The python likelihood tools are a very powerful set of analysis tools that expand upon the command line tools provided with the Fermi Science Tools package. Not only can you perform all of the same likelihood analysis with the python tools that you can with the standard command line tools but you can directly access all of the model parameters.  You can more easily script a standard analysis like light curve generation.  There are also a few things built into the python tools that are not available from the command line like the calculation of upper limits.

There are many [user contributed packages](/ssc/data/analysis/user/) built upon the python backbone of the Science Tools and we are going to highlight and use a few of them in this tutorial like likeSED, make3FGLxml, and the LATAnalysisScripts.

This sample analysis is based on the PG 1553+113 analysis performed by the LAT team and described in [Abdo, A. A. et al. 2010, ApJ, 708, 1310](http://adsabs.harvard.edu/abs/2010ApJ...708.1310A). At certain points we will refer to this article as well as the [Cicerone](/ssc/data/analysis/documentation/Cicerone/). After you complete this tutorial you should be able to reproduce all of the data analysis performed in this publication including generating a spectrum (individual bins and a butterfly plot) and produce a light curve with the python tools.  This tutorial assumes you have the most recent [ScienceTools](/ssc/data/analysis/software/) installed. We will also make significant use of python, so you might want to familiarize yourself with python (there's a beginner's guide at http://wiki.python.org/moin/BeginnersGuide. This tutorial also assumes that you've gone through the non-python based [unbinned likelihood](/ssc/data/analysis/scitools/likelihood_tutorial.html) thread. This tutorial should take approximately 8 hours to complete (depending on your computer's speed) if you do everything but there are some steps you can skip along the way which shave off about 4 hours of that.

> **Note:** In this tutorial, commands that start with `[user@localhost]$` are to be executed within a shell environment (bash, c-shell etc.) while commands that start with `>>>` are to be executed within an interactive python environment. Commands are in **bold** and outputs are in normal font.  Text in *ITALICS* within the boxes are comments from the author.

## Get the Data

For this thread the original data were extracted from the [LAT data server](/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the paper):

* Search Center (RA,Dec) = (238.929,11.1901)
* Radius = 20 degrees
* Start Time (MET) = 239557417 seconds (2008-08-04T15:43:37)
* Stop Time (MET) = 256970880 seconds (2009-02-22T04:48:00)
* Minimum Energy = 100 MeV
* Maximum Energy = 300000 MeV

We've provided direct links to the event files as well as the spacecraft data file if you don't want to take the time to use the download server. For more information on how to download LAT data please see the [Extract LAT Data](/ssc/data/analysis/scitools/extract_latdata.html) tutorial.

* [L1504241622054B65347F25_PH00.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L1504241622054B65347F25_PH00.fits)
* [L1504241622054B65347F25_PH01.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L1504241622054B65347F25_PH01.fits)
* [L1504241622054B65347F25_SC00.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/pyLikelihood/L1504241622054B65347F25_SC00.fits)

<p>
You'll first need to make a file list with the names of your input event files:
</p>
<code>
[user@localhost]$ <strong>ls -1 *PH*.fits &gt; PG1553_events.list</strong>
</code>
<p>
In the following analysis we've assumed that you've named your list of data files PG1553_events.list and the spacecraft file PG1553_SC.fits.
</p>